In [1]:
import numpy as np
from os.path import join, isfile
from skimage.measure import regionprops_table
import pandas as pd
from os.path import join
import inducedRainbowAnalysis
import os
from os import listdir
import random

In [53]:
clusterPath = '/Volumes/Aortas/Aorta/results/inducedRainbow/allClusterSizesRed.csv'
nucPath = "/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/nuclei"

In [3]:
dfResult = pd.read_csv(clusterPath)

In [60]:
dfStats = pd.DataFrame(columns = ['age', 'aorta', 'positiveCells', 'dividingCells', 'totalCells', 'dividingClusters', ])
for index, row in dfResult.iterrows():
    age = row["age"]
    aorta = row["aorta"]
    nucleiPath = join(nucPath, age.split("P")[1], aorta)
    positiveCells, dividedCells, positiveClusters, dividedClusters, nucNumber = 0,0,0,0,0
    numberOfClusters = row[2:17].sum()
    numberDivided = row[3:17].sum()
    clusterSize = []
    clusterSizeWOSingletons = []
    clusterWOSingletons = []
    files = [f for f in listdir(nucleiPath) if isfile(join(nucleiPath, f)) if ".DS_Store" not in f if "_seg.npy" in f if "._" not in f]
    for file in files:
        nucNumber += len(np.unique(np.load(join(nucleiPath, file), allow_pickle=True).item()["masks"])) -1
    for i in range(2, 17):
        cluster = np.full(row[i], i-1)
        positiveCells += row[i] * (i-1)
        positiveClusters += row[i]
        dividedCells += row[i] * (i-2)
        if i >2:
            clusterWOSingletons = np.full(row[i], i-1)
            dividedClusters += row[i]
        clusterSize = np.concatenate([clusterSize, cluster])
        clusterSizeWOSingletons = np.concatenate([clusterSizeWOSingletons, clusterWOSingletons])

    avgCloneSizeWithSingletons = np.mean(clusterSize)
    avgCloneSizeWOSingletons = np.mean(clusterSizeWOSingletons)
    cvCloneSizeWithSingletons = np.mean(clusterSize)/np.std(clusterSize)
    cvCloneSizeWOSingletons = np.mean(clusterSizeWOSingletons)/np.std(clusterSizeWOSingletons)
    df_help = pd.DataFrame([[age, aorta, positiveCells, dividedClusters, positiveClusters, dividedCells, nucNumber, avgCloneSizeWithSingletons, avgCloneSizeWOSingletons, cvCloneSizeWithSingletons, cvCloneSizeWOSingletons ]],columns = ['age', 'aorta', 'positiveCells', 'dividingClusters', 'marked', 'dividingCells', 'totalCells', 'avgCloneSize', 'avgCloneSizeWOSingleton', 'cvCloneSize', 'cvCloneSizeWOSingleton'] )
    dfStats = pd.concat([dfStats, df_help], axis=0)

/var/folders/6r/4p5g0w_j4jv11sr24nv1b0140000gn/T/ipykernel_1737/2847880887.py:29: RuntimeWarning: divide by zero encountered in double_scalars
  cvCloneSizeWOSingletons = np.mean(clusterSizeWOSingletons)/np.std(clusterSizeWOSingletons)
/var/folders/6r/4p5g0w_j4jv11sr24nv1b0140000gn/T/ipykernel_1737/2847880887.py:29: RuntimeWarning: divide by zero encountered in double_scalars
  cvCloneSizeWOSingletons = np.mean(clusterSizeWOSingletons)/np.std(clusterSizeWOSingletons)
/var/folders/6r/4p5g0w_j4jv11sr24nv1b0140000gn/T/ipykernel_1737/2847880887.py:29: RuntimeWarning: divide by zero encountered in double_scalars
  cvCloneSizeWOSingletons = np.mean(clusterSizeWOSingletons)/np.std(clusterSizeWOSingletons)


In [7]:
dfStats.to_csv("/Users/jones/Downloads/Stats.csv")

In [8]:
dfStats = pd.read_csv("/Users/jones/Downloads/Stats.csv", index_col=0)

In [9]:
dfStats.reset_index(drop = True)

,age,aorta,positiveCells,dividingCells,totalCells,dividingClusters,marked,avgCloneSize,avgCloneSizeWOSingleton,cvCloneSize,cvCloneSizeWOSingleton
0,P0-5,aorta01,710,186,9784,115,524.0,1.354962,2.617391,1.667512,2.662154
1,P0-5,aorta02,578,157,7741,94,421.0,1.372922,2.670213,1.598785,2.505369
2,P0-5,aorta03,620,230,6502,120,390.0,1.589744,2.916667,1.409164,2.310848
3,P0-5,aorta04,788,303,9045,170,485.0,1.624742,2.782353,1.386804,2.044174
4,P0-5,aorta05,512,175,6992,109,337.0,1.519288,2.605505,1.583751,2.482836
...,...,...,...,...,...,...,...,...,...,...,...
56,P5-60,aorta04,1136,601,16476,298,535.0,2.123364,3.016779,1.344766,1.844783
57,P5-60,aorta05,844,419,13292,233,425.0,1.985882,2.798283,1.534383,2.216047
58,P5-60,aorta06,1015,523,14401,285,492.0,2.063008,2.835088,1.463949,1.999020
59,P5-60,aorta07,399,152,12499,100,247.0,1.615385,2.520000,1.538055,2.169192


In [10]:
savePath = '/Volumes/Aortas/Aorta/results/inducedRainbow/markovianClusterNew'

In [12]:
for i, row in dfStats.iterrows():
    aorta = row['aorta']
    age = row['age']
    divCells = int(dfStats[(dfStats.age == age) & (dfStats.aorta == aorta)]['dividingCells'])
    marked = int(dfStats[(dfStats.age == age) & (dfStats.aorta == aorta)]['marked'])
    random.seed(42)
    start = list(range(1, marked + 1))
    resultsRandomCluster = []
    for i in range(1000):
        randomCluster = start.copy()
        for i in range(divCells):
            randomCluster.append(random.randrange(1, marked + 1))
        clusterSizesRandomCluster = []
        for i in range(1,marked +1):
            clusterSizesRandomCluster.append(randomCluster.count(i))
        resultsRandomCluster.append(clusterSizesRandomCluster)

    random.seed(42)
    resultsRandomDivision = []
    for i in range(1000):
        randomDivision = start.copy()
        for j in range(divCells):
            randomDivision.append(random.choice(randomDivision))
        clusterSizesRandomDivision = []
        for i in range(1,marked + 1):
            clusterSizesRandomDivision.append(randomDivision.count(i))
        resultsRandomDivision.append(clusterSizesRandomDivision)

    os.chdir(savePath)
    if not os.path.exists(join(savePath, age)):
        os.mkdir(age)
    os.chdir(join(savePath, age))
    if not os.path.exists(join(savePath, age, aorta)):
        os.mkdir(aorta)
    os.chdir(join(savePath, age, aorta))

    filenameCluster = "markovianCluster_" + age + "_" + aorta + ".txt"
    filenameDivision = "markovianDivision_" + age + "_" + aorta + ".txt"

    np.savetxt(join(savePath, age, aorta, filenameCluster), resultsRandomCluster)
    np.savetxt(join(savePath, age, aorta, filenameDivision), resultsRandomDivision)

In [19]:
dfCopy = dfStats.loc[(dfStats.age == "P0-5") & (dfStats.aorta == "aorta01")]
dfCopy

,age,aorta,positiveCells,dividingCells,totalCells,dividingClusters,marked,avgCloneSize,avgCloneSizeWOSingleton,cvCloneSize,cvCloneSizeWOSingleton
0,P0-5,aorta01,710,186,9784,115,524.0,1.354962,2.617391,1.667512,2.662154


In [47]:
maxCluster, maxDiv = 15, 21
dfDiv = pd.DataFrame()
dfCluster = pd.DataFrame()

for index, row in dfStats.iterrows():
    aorta = row['aorta']
    age = row['age']
    divCells = int(dfStats[(dfStats.age == age) & (dfStats.aorta == aorta)]['dividingCells'])
    marked = int(dfStats[(dfStats.age == age) & (dfStats.aorta == aorta)]['marked'])
    random.seed(42)
    start = list(range(1, marked + 1))
    resultsRandomCluster = []
    for i in range(1000):
        randomCluster = start.copy()
        for j in range(divCells):
            randomCluster.append(random.randrange(1, marked + 1))
        clusterSizesRandomCluster = []
        for m in range(1,marked +1):
            clusterSizesRandomCluster.append(randomCluster.count(m))
        resultsRandomCluster.append(clusterSizesRandomCluster)

        resultdic = {"age": age, "aorta": aorta, "simulation": i+1}
        unique, counts = np.unique(clusterSizesRandomCluster, return_counts=True)
        for m in range(1,maxCluster+1):
            if m in unique:
                resultdic[m] = counts[np.where(unique==m)][0]
            else:
                resultdic[m] = 0
        dfHelp = pd.DataFrame(resultdic, index = [0])
        dfCluster = pd.concat([dfCluster, dfHelp])

    random.seed(42)
    resultsRandomDivision = []
    for i in range(1000):
        randomDivision = start.copy()
        for j in range(divCells):
            randomDivision.append(random.choice(randomDivision))
        clusterSizesRandomDivision = []
        for m in range(1,marked + 1):
            clusterSizesRandomDivision.append(randomDivision.count(m))
        resultsRandomDivision.append(clusterSizesRandomDivision)

        resultdic = {"age": age, "aorta": aorta, "simulation": i+1}
        unique, counts = np.unique(clusterSizesRandomDivision, return_counts=True)
        for m in range(1,maxDiv+1):
            if m in unique:
                resultdic[m] = counts[np.where(unique==m)][0]
            else:
                resultdic[m] = 0
        dfHelp = pd.DataFrame(resultdic, index = [0])
        dfDiv = pd.concat([dfDiv, dfHelp])

In [50]:
dfCluster

,age,aorta,simulation,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,P0-5,aorta01,1,375,116,29,4,0,0,0,0,0,0,0,0,0,0,0
0,P0-5,aorta01,2,370,126,25,2,1,0,0,0,0,0,0,0,0,0,0
0,P0-5,aorta01,3,363,137,23,1,0,0,0,0,0,0,0,0,0,0,0
0,P0-5,aorta01,4,359,145,19,1,0,0,0,0,0,0,0,0,0,0,0
0,P0-5,aorta01,5,362,142,16,4,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,P5-60,aorta08,996,194,150,55,14,2,1,0,0,0,0,0,0,0,0,0
0,P5-60,aorta08,997,196,144,58,17,1,0,0,0,0,0,0,0,0,0,0
0,P5-60,aorta08,998,193,156,45,19,3,0,0,0,0,0,0,0,0,0,0
0,P5-60,aorta08,999,204,138,53,16,2,3,0,0,0,0,0,0,0,0,0


In [53]:
dfClusterSum = dfCluster.groupby(["age", "aorta"]).sum()
dfDivSum = dfDiv.groupby(["age", "aorta"]).sum()

In [54]:
dfCluster.to_csv("/Users/jones/Downloads/markovianClusterAllClusterSizes.csv")
dfClusterSum.to_csv("/Users/jones/Downloads/markovianClusterAllClusterSizesSum.csv")
dfDiv.to_csv("/Users/jones/Downloads/markovianDivisionAllClusterSizes.csv")
dfDivSum.to_csv("/Users/jones/Downloads/markovianDivisionAllClusterSizesSum.csv")

In [31]:
maxDiv

21

In [32]:
maxCluster

15

In [ ]:
resultdic = {"age": segmentationPath.split("/")[-3], "aorta": segmentationPath.split("/")[-2]}
unique, counts = np.unique(clusterSizes, return_counts=True)
for i in range(1,18):
    if i in unique:
        resultdic[i] = counts[np.where(unique==i)][0]
    else:
        resultdic[i] = 0

In [33]:
resultsRandomDivision[0]

[2,
 1,
 2,
 7,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 3,
 2,
 2,
 1,
 8,
 5,
 1,
 1,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 2,
 1,
 1,
 3,
 2,
 2,
 1,
 4,
 4,
 1,
 1,
 2,
 1,
 3,
 3,
 3,
 3,
 1,
 3,
 3,
 1,
 3,
 1,
 2,
 1,
 3,
 3,
 2,
 3,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 3,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 3,
 2,
 3,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 3,
 3,
 7,
 1,
 6,
 1,
 5,
 3,
 1,
 3,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 4,
 1,
 1,
 2,
 1,
 1,
 1,
 6,
 1,
 1,
 1,
 1,
 3,
 2,
 6,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 2,
 1,
 1,
 3,
 1,
 2,
 1,
 2,
 1,
 4,
 2,
 2,
 2,
 1,
 3,
 1,
 3,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 3,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 10,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 5,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 3,
 2,
 1,
 1,
 1,
 3,
 2,
 1,
 2,
 1,
 4,
 3,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 3,
 2,
 1,
 2,
 1,

In [ ]:
for simul in resultsRandomDivision:


In [24]:
unique, counts = np.unique(resultsRandomDivision[0], return_counts=True)

In [25]:
unique

array([1, 2, 3, 4, 5, 7])

In [27]:
np.max(unique)

7

In [26]:
counts

array([396,  88,  27,  10,   2,   1])

In [20]:
dfResult[[""]].sum(axis = 1)

SyntaxError: invalid syntax (3901764732.py, line 1)

In [26]:
dfResult.iloc[:,2:17].sum(axis=1)

0     524
1     421
2     390
3     485
4     337
     ... 
56    535
57    425
58    492
59    247
60    416
Length: 61, dtype: int64

In [40]:
dfResult.iloc[0]

age         P0-5
aorta    aorta01
1            409
2             71
3             27
4             11
5              3
6              2
7              1
8              0
9              0
10             0
11             0
12             0
13             0
14             0
15             0
Name: 0, dtype: object

In [52]:
for index, row in dfResult.iterrows():
    age = row["age"]
    aorta = row["aorta"]
    print(age.split("P")[1])

0-5
0-5
0-5
0-5
0-5
0-5
0-10
0-10
0-10
0-10
0-10
0-10
10-21
10-21
10-21
10-21
10-21
10-30
10-30
10-30
10-30
10-30
10-30
10-30
10-60
10-60
10-60
10-60
10-60
10-60
21-30
21-30
21-30
21-30
21-30
30-60
30-60
30-60
30-60
30-60
5-10
5-10
5-10
5-10
5-10
5-10
5-10
5-30
5-30
5-30
5-30
5-30
5-30
5-60
5-60
5-60
5-60
5-60
5-60
5-60
5-60


In [50]:
clusterSize

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1